In [1]:
# Initialize PaddleOCR instance
from paddleocr import PaddleOCR
import cv2
import numpy as np
import regex as re
from deskew import determine_skew
import os

# paddleocr model for detection and extraction of text
ocr = PaddleOCR(lang = 'en')

C:\Users\hp333\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\hp333\AppData\Roaming\Python\Python311\site-packages\paddle\utils\cpp_extension\extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\hp333\.paddlex\official_models\PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\hp333\.paddlex\official_models\UV

In [2]:
def needs_opening(img):
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()

    if len(np.unique(gray)) == 2:  # Already binary
        bw = gray
    else:
        _, bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(bw, connectivity=8)
    if num_labels < 2:
        return False

    img_area = bw.shape[0] * bw.shape[1]

    small_objects = sum(
        1 for area in stats[1:, cv2.CC_STAT_AREA]  # Skip background (index 0)
        if area < (img_area * 0.0001)  # Less than 0.01% of image
    )
    noise_threshold = max(20, num_labels * 0.1)  # At least 20 or 10% of objects
    return small_objects > noise_threshold


def needs_closing(img):
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()
    edges = cv2.Canny(gray, 50, 150)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    grad = cv2.morphologyEx(gray, cv2.MORPH_GRADIENT, kernel)
    edge_density = np.count_nonzero(edges) / edges.size
    grad_std = np.std(grad)

    return edge_density > 0.15 or grad_std > 40


def needs_dilation(img):
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()

    if len(np.unique(gray)) == 2:
        bw = gray
    else:
        _, bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    if np.mean(bw) > 127:
        bw = cv2.bitwise_not(bw)
    dist = cv2.distanceTransform(bw, cv2.DIST_L2, 5)
    text_pixels = dist[dist > 0]

    if len(text_pixels) == 0:  # No text found
        return False

    mean_stroke = np.mean(text_pixels)
    median_stroke = np.median(text_pixels)
    min_stroke = max(1.5, min(bw.shape) * 0.003)
    return mean_stroke < min_stroke or median_stroke < 1.0

In [3]:
def intelligent_dpi_adjustment(img):
    """
    Complete intelligent DPI adjustment for OCR.
    Combines multiple strategies for best results.
    """
    h, w = img.shape[:2]
    print(f"\n{'='*50}")
    print(f"Original dimensions: {w}x{h}")

    # Strategy 1: Check if image is reasonable size
    min_dimension = min(w, h)
    max_dimension = max(w, h)

    # Rule 1: Minimum size check (for very small images)
    if max_dimension < 800:
        scale = 800 / max_dimension
        print(f"Rule: Image too small, scaling up by {scale:.2f}x")

    # Rule 2: Maximum size check (for very large images)
    elif max_dimension > 4000:
        scale = 4000 / max_dimension
        print(f"Rule: Image too large, scaling down by {scale:.2f}x")

    # Rule 3: Optimal range (1200-2500px on longest edge)
    elif max_dimension < 1200:
        scale = 1500 / max_dimension
        print(f"Rule: Upscaling to optimal range ({scale:.2f}x)")
    elif max_dimension > 2500:
        scale = 2000 / max_dimension
        print(f"Rule: Downscaling to optimal range ({scale:.2f}x)")
    else:
        scale = 1.0
        print(f"Rule: Size already optimal, no scaling needed")

    # Apply scaling
    if scale != 1.0:
        new_w = int(w * scale)
        new_h = int(h * scale)

        # Choose best interpolation
        if scale > 1.0:
            interpolation = cv2.INTER_CUBIC  # Smooth upscaling
            print("Interpolation: INTER_CUBIC (upscaling)")
        else:
            interpolation = cv2.INTER_AREA  # Sharp downscaling
            print("Interpolation: INTER_AREA (downscaling)")

        img = cv2.resize(img, (new_w, new_h), interpolation=interpolation)
        print(f"Final dimensions: {new_w}x{new_h}")
        print(f"Equivalent DPI: ~{new_h / 11 * 300:.0f} DPI (assuming 11\" height)")

    print(f"{'='*50}\n")
    return img

In [6]:
#preprocessing pipeline
path = "C:/Users/hp333/Desktop/ocr/Data/reverseWaybill-156387426414724544_1.jpg"
img = cv2.imread(path)

h, w = img.shape[:2]
print("height : ", h, " width : ", w)

gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
angle = determine_skew(gray_img)
(h, w) = gray_img.shape
M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
deskewed = cv2.warpAffine(gray_img, M, (w, h), borderMode=cv2.BORDER_REPLICATE)
# img = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 11, 2)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
img = clahe.apply(gray_img)

#smoothing
noise_level = np.std(img)
print("Noise level: ", noise_level)
if noise_level > 35:
    img = cv2.fastNlMeansDenoising(img, None, h=12, templateWindowSize=7, searchWindowSize=21)
    print("smoothing done")
elif noise_level > 25:
    img = cv2.GaussianBlur(img, (3, 3), 0)
    print("smoothing done")

# sharpening using kernels
laplacian_var = cv2.Laplacian(gray_img, cv2.CV_64F).var()
print("Laplacian variance:", laplacian_var)

# Adaptive threshold based on image size
img_size = w * h
adaptive_threshold = 100 * (img_size / 1000000)  # Scale with megapixels
adaptive_threshold = max(50, min(adaptive_threshold, 500))  # Clamp between 50-500

print(f"Adaptive blur threshold: {adaptive_threshold:.1f}")

if laplacian_var < adaptive_threshold:
    print("Sharpening applied")
    # Unsharp masking (gentler than direct kernel)
    gaussian = cv2.GaussianBlur(img, (0, 0), 2.0)
    img = cv2.addWeighted(img, 1.5, gaussian, -0.5, 0)

#morphological cleanup
kernel_size = max(3, min(w, h) // 200)
opening_kernel = np.ones((max(3, min(w,h)//300), max(3, min(w,h)//300)), np.uint8)
closing_kernel = np.ones((max(5, min(w,h)//150), max(5, min(w,h)//150)), np.uint8)
if needs_dilation(img):
    print("dilation done")
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    img = cv2.dilate(img, kernel)
    # img = cv2.dilate(img, kernel_size)

if needs_opening(img):
    print("opening done")
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, opening_kernel)

if needs_closing(img):
    print("closing done")
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, closing_kernel)

#DPI adjustments
img = intelligent_dpi_adjustment(img)
final_image = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

# Results
cv2.imshow("preprocessed image", final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

height :  2160  width :  3840
Noise level:  38.04420779780377
smoothing done
Laplacian variance: 106.84604515594019
Adaptive blur threshold: 500.0
Sharpening applied
opening done

Original dimensions: 3840x2160
Rule: Downscaling to optimal range (0.52x)
Interpolation: INTER_AREA (downscaling)
Final dimensions: 2000x1125
Equivalent DPI: ~30682 DPI (assuming 11" height)



In [7]:
# Run OCR inference on a sample image
result = ocr.predict(
    input = final_image)

In [6]:
import json
def text_from_json(json_data):
    """
    Extract and fix barcode text directly from PaddleOCR JSON output
    No need to run OCR again!
    """
    # Parse JSON if it's a string
    if isinstance(json_data, str) and os.path.isfile(json_data):
        with open(json_data, "r", encoding="utf-8") as f:
            data = json.load(f)
    else:
        # Otherwise treat the string as raw JSON text
        data = json.loads(json_data) if isinstance(json_data, str) else json_data

    # Get all recognized texts and their scores
    rec_texts = data.get('rec_texts', [])
    rec_scores = data.get('rec_scores', [])
    rec_boxes = data.get('rec_boxes', [])

    results = []

    # Process each detected text
    for i, text in enumerate(rec_texts):
        confidence = rec_scores[i] if i < len(rec_scores) else 0.0
        bbox = rec_boxes[i] if i < len(rec_boxes) else None

        # Try to fix underscore pattern
        fixed_text = fix_underscore_pattern(rec_texts[i])

        if fixed_text:
            results.append({
                'original': text,
                'fixed': fixed_text,
                'confidence': confidence,
                'bbox': bbox
            })

    return results

def fix_underscore_pattern(text):
    """
    Fix missing underscores in barcode text
    Expected pattern: 156381A26414724544_1_whl
    Format: [alphanumeric sequence]_[digit]_[letters]
    """
    # Remove all spaces first
    text_no_spaces = text.replace(' ', '')

    # If already has underscores in correct pattern, return as is
    if re.match(r'^[0-9A-Z]+_\d+_[a-zA-Z]+$', text_no_spaces, re.IGNORECASE):
        return text_no_spaces

    # Pattern 1: Long alphanumeric + digit + letters (156381A26414724544 1 whl)
    # Expected: XXXXXXXXXXXX_X_XXX
    pattern1 = r'^([0-9A-Z]{10,})\s*(\d{1,2})\s*([a-zA-Z]{2,5})$'
    match = re.match(pattern1, text, re.IGNORECASE)
    if match:
        return f"{match.group(1)}_{match.group(2)}_{match.group(3)}"

    # Pattern 2: If OCR merged everything without spaces
    # Try to split based on length (assuming 18 chars + 1 digit + 3 letters)
    pattern2 = r'^([0-9A-Z]{14,20})(\d{1,2})([a-zA-Z]{2,5})$'
    match = re.match(pattern2, text_no_spaces, re.IGNORECASE)
    if match:
        return f"{match.group(1)}_{match.group(2)}_{match.group(3)}"

    # Pattern 3: Only one underscore present (156381A26414724544_1whl or 156381A264147245441_whl)
    if text.count('_') == 1:
        # Case: XXXXX_Xwhl (missing second underscore)
        pattern3a = r'^([0-9A-Z]+)_(\d{1,2})([a-zA-Z]{2,5})$'
        match = re.match(pattern3a, text, re.IGNORECASE)
        if match:
            return f"{match.group(1)}_{match.group(2)}_{match.group(3)}"

        # Case: XXXXX1_whl (missing first underscore)
        pattern3b = r'^([0-9A-Z]+?)(\d{1,2})_([a-zA-Z]{2,5})$'
        match = re.match(pattern3b, text, re.IGNORECASE)
        if match:
            return f"{match.group(1)}_{match.group(2)}_{match.group(3)}"

    # Pattern 4: Has spaces that should be underscores
    # Replace spaces with underscores if text looks like barcode format
    if re.match(r'^[0-9A-Z\s]+\d+\s+[a-zA-Z]+$', text, re.IGNORECASE):
        # Split by space and reconstruct
        parts = text.split()
        if len(parts) >= 3:
            # Last part is letters, second last is digit(s)
            return '_'.join(parts)

    return None

In [9]:
# Visualize the results and save the JSON results
temp = 1
valid_results = []
output_dir = "../results/output"
os.makedirs(output_dir, exist_ok=True)
for res in result:
    image = res.save_to_img(save_dir = output_dir, file_name = f"result_img_{temp}.png")
    # out_path = os.path.join(output_dir, f"result_img_{temp}.png")
    # cv2.imwrite(out_path, res.img)
    json_file = res.save_to_json(save_dir = output_dir, file_name = f"result_json_{temp}.json")
    # out_path = os.path.join(output_dir, f"result_json_{temp}.json")
    # cv2.imwrite(out_path, res.json)
    temp += 1

error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'imwrite'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'


In [12]:
results = text_from_json(result)
# Expected pattern: [long alphanumeric]_[1-2 digits]_[2-5 letters]
expected_pattern = r'^[0-9A-Z]{14,20}_\d{1,2}_[a-zA-Z]{2,5}$'

for result in results:
    fixed_text = result['fixed']
    if fixed_text and re.match(expected_pattern, fixed_text, re.IGNORECASE):
        valid_results.append(result)
print(valid_results)

AttributeError: 'list' object has no attribute 'get'

In [4]:
path = "/Data/reverseWaybill-156387426414724544_1.jpg"
img = cv2.imread(path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

angle = determine_skew(gray)
(h, w) = gray.shape
M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
deskewed = cv2.warpAffine(gray, M, (w, h), borderMode=cv2.BORDER_REPLICATE)

cv2.imwrite("deskewed.jpg", deskewed)


True